In [14]:
# Importing the necessary library's

import pandas as pd
import base64
import os
from requests import get, post
import json
import time

In [3]:
# Reading the dataset and creating a dataframe

df = pd.read_csv('charts.csv')
print(df.head(3))

                         title  rank        date        artist  \
0      Chantaje (feat. Maluma)     1  2017-01-01       Shakira   
1  Vente Pa' Ca (feat. Maluma)     2  2017-01-01  Ricky Martin   
2   Reggaetón Lento (Bailemos)     3  2017-01-01          CNCO   

                                                 url     region   chart  \
0  https://open.spotify.com/track/6mICuAdrwEjh6Y6...  Argentina  top200   
1  https://open.spotify.com/track/7DM4BPaS7uofFul...  Argentina  top200   
2  https://open.spotify.com/track/3AEZUABDXNtecAO...  Argentina  top200   

           trend   streams  
0  SAME_POSITION  253019.0  
1        MOVE_UP  223988.0  
2      MOVE_DOWN  210943.0  


In [4]:
# Filtering the dataframe and using the country United States only

df = df[df['region'] == 'United States']
print(len(df))

455057


In [5]:
print(df.head())

                                    title  rank        date  \
8246  Bad and Boujee (feat. Lil Uzi Vert)     1  2017-01-01   
8247                            Fake Love     2  2017-01-01   
8248                              Starboy     3  2017-01-01   
8249                               Closer     4  2017-01-01   
8250                        Black Beatles     5  2017-01-01   

                        artist  \
8246                     Migos   
8247                     Drake   
8248     The Weeknd, Daft Punk   
8249  The Chainsmokers, Halsey   
8250  Rae Sremmurd, Gucci Mane   

                                                    url         region  \
8246  https://open.spotify.com/track/4Km5HrUvYTaSUfi...  United States   
8247  https://open.spotify.com/track/343YBumqHu19cGo...  United States   
8248  https://open.spotify.com/track/5aAx2yezTd8zXrk...  United States   
8249  https://open.spotify.com/track/7BKLCZ1jbUBVqRi...  United States   
8250  https://open.spotify.com/track/6fujklziT

In [6]:
# Droppinf duplicate values

df = df.drop_duplicates(subset=['title'], keep='first')
print(len(df))

11842


In [7]:
# Creating a column 'track_id'

df['title_id'] = df['url'].str[31:]
print(df.head())

                                    title  rank        date  \
8246  Bad and Boujee (feat. Lil Uzi Vert)     1  2017-01-01   
8247                            Fake Love     2  2017-01-01   
8248                              Starboy     3  2017-01-01   
8249                               Closer     4  2017-01-01   
8250                        Black Beatles     5  2017-01-01   

                        artist  \
8246                     Migos   
8247                     Drake   
8248     The Weeknd, Daft Punk   
8249  The Chainsmokers, Halsey   
8250  Rae Sremmurd, Gucci Mane   

                                                    url         region  \
8246  https://open.spotify.com/track/4Km5HrUvYTaSUfi...  United States   
8247  https://open.spotify.com/track/343YBumqHu19cGo...  United States   
8248  https://open.spotify.com/track/5aAx2yezTd8zXrk...  United States   
8249  https://open.spotify.com/track/7BKLCZ1jbUBVqRi...  United States   
8250  https://open.spotify.com/track/6fujklziT

In [8]:
# Spotify Credentials for generating token

client_id = "c392d42a11bf414182f69a9e40526bc6"
client_secret = "fde1f03252d749319749f55473ee2247"

print(client_id, client_secret)

def get_token():
      auth_string = client_id + ":" + client_secret
      auth_bytes = auth_string.encode("utf-8")
      auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
      url = "https://accounts.spotify.com/api/token"
      headers = {
          "Authorization": "Basic " + auth_base64,
          "Content-Type": "application/x-www-form-urlencoded"
      }
      data = {"grant_type": "client_credentials"}
      result = post(url, headers=headers, data=data)
      json_result = json.loads (result.content)
      token = json_result["access_token"]
      return token

def get_auth_header (token):
    return {"Authorization": "Bearer " + token}

c392d42a11bf414182f69a9e40526bc6 fde1f03252d749319749f55473ee2247


In [9]:
# getting Audio Features Function

i= 0

def get_track_audio_features(token, track_id):
    query_url = 'https://api.spotify.com/v1/audio-features?ids=' + track_id
    headers = get_auth_header(token)
    try:
        result = get(query_url, headers=headers)
    except ConnectionError as ce:
        i = i + 1
        print(f"Time limit exceeded for the {i}th time")
        time.sleep(120)
        token = get_token()
        return get_track_audio_features(token, track_id)
    if result.status_code != 200:
        print("Error Received for the following query: ")
        print("\n")
        print(query_url)
        print("\n")
        return ""
        
    json_result = json.loads(result.content)
    return json_result

In [12]:
# Concatenating track_ID's to reduce the no of API calls, by sending 50 ID's in 1 API call.

concatenated_track_ids = []

for i in range(0, len(df['title_id']), 100):
    concatenated_ids = ','.join(df['title_id'].iloc[i:i+100])
    concatenated_track_ids.append(concatenated_ids)

print(len(concatenated_track_ids))

119


In [15]:
# Creating a list for audio features of all the title_id's

audio_features_list = []

for s in concatenated_track_ids:
    token = get_token()
    obj = get_track_audio_features(token, s)
    for item in obj['audio_features']:
        audio_features_list.append(item)

print(len(audio_features_list))

11842


In [26]:
print(audio_features_list[0])

{'danceability': 0.926, 'energy': 0.666, 'key': 11, 'loudness': -5.314, 'mode': 1, 'speechiness': 0.244, 'acousticness': 0.0611, 'instrumentalness': 0, 'liveness': 0.123, 'valence': 0.168, 'tempo': 127.079, 'type': 'audio_features', 'id': '4Km5HrUvYTaSUfiSGPJeQR', 'uri': 'spotify:track:4Km5HrUvYTaSUfiSGPJeQR', 'track_href': 'https://api.spotify.com/v1/tracks/4Km5HrUvYTaSUfiSGPJeQR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Km5HrUvYTaSUfiSGPJeQR', 'duration_ms': 343150, 'time_signature': 4}


In [18]:
print(df.head())

                                    title  rank        date  \
8246  Bad and Boujee (feat. Lil Uzi Vert)   1.0  2017-01-01   
8247                            Fake Love   2.0  2017-01-01   
8248                              Starboy   3.0  2017-01-01   
8249                               Closer   4.0  2017-01-01   
8250                        Black Beatles   5.0  2017-01-01   

                        artist  \
8246                     Migos   
8247                     Drake   
8248     The Weeknd, Daft Punk   
8249  The Chainsmokers, Halsey   
8250  Rae Sremmurd, Gucci Mane   

                                                    url         region  \
8246  https://open.spotify.com/track/4Km5HrUvYTaSUfi...  United States   
8247  https://open.spotify.com/track/343YBumqHu19cGo...  United States   
8248  https://open.spotify.com/track/5aAx2yezTd8zXrk...  United States   
8249  https://open.spotify.com/track/7BKLCZ1jbUBVqRi...  United States   
8250  https://open.spotify.com/track/6fujklziT

In [30]:
# Creating new columns in the df and populating them with their corresponding values from audio_features_list

for i, obj in enumerate(audio_features_list):
    # print(f"For {i} -> {obj}")
    df['danceability'].iloc[i] = obj['danceability']
    df['energy'].iloc[i] = obj['energy']
    df['key'].iloc[i] = obj['key']
    df['loudness'].iloc[i] = obj['loudness']
    df['mode'].iloc[i] = obj['mode']
    df['speechiness'].iloc[i] = obj['speechiness']
    df['acousticness'].iloc[i] = obj['acousticness']
    df['instrumentalness'].iloc[i] = obj['instrumentalness']
    df['liveness'].iloc[i] = obj['liveness']
    df['valence'].iloc[i] = obj['valence']
    df['tempo'].iloc[i] = obj['tempo']
    df['duration_ms'].iloc[i] = obj['duration_ms']
    df['time_signature'].iloc[i] = obj['time_signature']

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
print(df.head(3))

                                    title  rank        date  \
8246  Bad and Boujee (feat. Lil Uzi Vert)   1.0  2017-01-01   
8247                            Fake Love   2.0  2017-01-01   
8248                              Starboy   3.0  2017-01-01   

                     artist  \
8246                  Migos   
8247                  Drake   
8248  The Weeknd, Daft Punk   

                                                    url         region  \
8246  https://open.spotify.com/track/4Km5HrUvYTaSUfi...  United States   
8247  https://open.spotify.com/track/343YBumqHu19cGo...  United States   
8248  https://open.spotify.com/track/5aAx2yezTd8zXrk...  United States   

       chart          trend    streams                title_id  ... liveness  \
8246  top200  SAME_POSITION  1371493.0  4Km5HrUvYTaSUfiSGPJeQR  ...    0.123   
8247  top200  SAME_POSITION  1180074.0  343YBumqHu19cGoGARUTsd  ...    0.176   
8248  top200  SAME_POSITION  1064351.0  5aAx2yezTd8zXrkmtKl66Z  ...    0.134   

    

In [32]:
# Creating a csv file for the same

df.to_csv('Final_Data.csv', index=False)